In [16]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss
import matplotlib.pyplot as plt
import plotly.graph_objs as go
%matplotlib inline

In [17]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas import datetime
from pmdarima.arima import auto_arima


C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



## Load Dataset

In [18]:
data = pd.read_csv('AirPassengers.csv', index_col =[0])
data.head()

,#Passengers
Month,
1949-01,112
1949-02,118
1949-03,132
1949-04,129
1949-05,121


In [19]:
data.reset_index(level=0, inplace=True)
data.columns = ['Month', 'Passengers']
data.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


## Initial Study

In [20]:
data.describe()

,Passengers
count,144.000000
mean,280.298611
std,119.966317
min,104.000000
25%,180.000000
50%,265.500000
75%,360.500000
max,622.000000


In [21]:
data.dtypes

Month         object
Passengers     int64
dtype: object

In [22]:
data.isnull().sum()

Month         0
Passengers    0
dtype: int64

In [23]:
data.nunique()

Month         144
Passengers    118
dtype: int64

## Vizsualizations

In [24]:
fig = go.Figure(go.Scatter(x = data.Month, y = data.Passengers, mode = 'markers+lines', name="Time Series"))
fig.update_layout(title='Original Dataset',showlegend=True, template="plotly_white", width=800, height=400)
fig.show()

In [25]:
# data['Year'] = pd.to_datetime(data.Month).dt.year
# print(data.head(3))
# print(data.tail(3))

In [26]:
# fmt = '{:<10}{:<10}{:<22}{}'
# print(fmt.format('', 'Year', 'Number of Months', 'Total Passengers'))
# for i, (yr, month, number) in enumerate(zip(data.groupby('Year')['Month'].count().index, data.groupby('Year')['Month'].count(), data.groupby('Year')['Passengers'].sum())):
#     print(fmt.format(i,yr, month, number))

## 1 .Check Stationarity of a Time Series
ADF Test

KPSS Test

Rolling Statistics (mean and std dev)

In [27]:
# Define Testing Functions

# For ADF Test
# null hypothesis: series is non-stationary
# If test statistic < critical value = reject the null hypothesis (i.e. series is stationary)
# If test statistic > critical value = accept the null hypothesis (i.e. series is not stationary)
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    

# For KPSS Test
# null hypothesis: series is stationary
# If test statistic < critical value = accept the null hypothesis (i.e. series is stationary)
# If test statistic > critical value = reject the null hypothesis (i.e. series is not stationary)
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)


# Rolling Statistics
def rolling_stat_test(timeseries, k =12):
    rolmean = timeseries.rolling(k).mean()
    rolstd = timeseries.rolling(k).std()
    
    fig = go.Figure([
            go.Scatter(y = timeseries.values,  mode = 'lines', name="Original"),
            go.Scatter(y = rolmean.values, mode = 'lines', name="Rolling Mean"),
            go.Scatter(y = rolstd.values, mode = 'lines', name="'Rolling Std'"),
        ])
    fig.update_layout(title='Rolling Mean & Standard Deviation',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
#     return
    
    
rolling_stat_test(data.Passengers)

In [28]:
adf_test(data.Passengers)

Results of Dickey-Fuller Test:
Test Statistic                   0.815369
p-value                          0.991880
Lags Used                       13.000000
Number of Observations Used    130.000000
Critical Value (1%)             -3.481682
Critical Value (5%)             -2.884042
Critical Value (10%)            -2.578770
dtype: float64


In [29]:
kpss_test(data.Passengers)

Results of KPSS Test:
Test Statistic            1.052175
p-value                   0.010000
Lags Used                14.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1661: FutureWarning:

The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'

C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning:

p-value is smaller than the indicated p-value



## 2. Make the Time- Series Stationary

In [30]:
# Strict Stationary: A strict stationary series satisfies the mathematical definition of a stationary process. 
# For a strict stationary series, the mean, variance and covariance are not the function of time. 
# The aim is to convert a non-stationary series into a strict stationary series for making predictions.

# Trend Stationary: A series that has no unit root but exhibits a trend is referred to as a trend stationary series.
# Once the trend is removed, the resulting series will be strict stationary. 
# The KPSS test classifies a series as stationary on the absence of unit root. 
# This means that the series can be strict stationary or trend stationary.

# Difference Stationary: A time series that can be made strict stationary by differencing falls under difference stationary. 
# ADF test is also known as a difference stationarity test.

In [31]:
# Case 1: Both tests conclude that the series is not stationary -> series is not stationary
# Case 2: Both tests conclude that the series is stationary -> series is stationary
# Case 3: KPSS = stationary and ADF = not stationary  -> trend stationary, remove the trend to make series strict stationary
# Case 4: KPSS = not stationary and ADF = stationary -> difference stationary, use differencing to make series stationary

### a. Differencing

In [32]:
# to remove varying mean
def differencing (timeseries):
    differencing = timeseries - timeseries.shift(1)
    fig = go.Figure(go.Scatter(y = differencing,  mode = 'markers+lines', name=""))
    fig.update_layout(title='Differencing ', showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
#     return differencing

differencing(data.Passengers)

### b. Seasonal Differencing

In [33]:
# to remove varying mean
def seasonal_differencing (timeseries, n=6):
    seasonal_differencing = timeseries - timeseries.shift(n)
    fig = go.Figure(go.Scatter(y = seasonal_differencing,  mode = 'markers+lines', name=""))
    fig.update_layout(title='seasonal_differencing ', showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
#     return seasonal_differencing

seasonal_differencing(data.Passengers,6)

### c. Transformation

In [34]:
# to stabilize the non-constant variance ie Reduce Trend
# power transform, square root, and log transform

def transformation (timeseries, mode='log'):
    if mode =='log':
        trans_log = np.log(timeseries)
        trans_diff = trans_log - trans_log.shift(1)
        
    if mode =='sqrt':
        trans_sqrt = np.sqrt(timeseries)
        trans_diff = trans_log - trans_log.shift(1)
        
    fig = go.Figure(go.Scatter(y = trans_diff,  mode = 'lines', name=mode +' transformation '))
    fig.update_layout(title= mode +' transformation ', showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
#     return trans_diff

transformation(data.Passengers)


In [35]:
# If trent is not very intuitive in presence of noise,we use some techniques to estimate 
# or model this trend and then remove it from the series. Ways are:

#     Aggregation – taking average for a time period like monthly/weekly averages
#     Smoothing – taking rolling averages
#     Polynomial Fitting – fit a regression model


### d. Moving average

In [36]:
def moving_average(timeseries, k =12):
    
    timeseries_log = np.log(timeseries)
    rolmean_log = timeseries_log.rolling(k).mean()
    timeseries_log_moving_avg_diff = timeseries_log - rolmean_log
    timeseries_log_moving_avg_diff.dropna(inplace=True)
    
    rolling_stat_test(timeseries_log_moving_avg_diff)
    print("----------")
    adf_test(timeseries_log_moving_avg_diff)
    print("----------")
    kpss_test(timeseries_log_moving_avg_diff)
#     return timeseries_log_moving_avg_diff

moving_average(data.Passengers)

----------
Results of Dickey-Fuller Test:
Test Statistic                  -3.162908
p-value                          0.022235
Lags Used                       13.000000
Number of Observations Used    119.000000
Critical Value (1%)             -3.486535
Critical Value (5%)             -2.886151
Critical Value (10%)            -2.579896
dtype: float64
----------
Results of KPSS Test:
Test Statistic            0.152538
p-value                   0.100000
Lags Used                13.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1661: FutureWarning:

The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'

C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning:

p-value is greater than the indicated p-value



In [37]:
# Disadvantage: time-period has to be strictly defined. 
# In complex situations like forecasting a stock price its difficult to come up with a number
# So we prefer: exponentially weighted moving average 

### e. Exponentially weighted moving average 

weights are assigned to all the previous values with a decay factor.

In [38]:
def exponentially_weighted_moving_average(timeseries, k =12):
    
    timeseries_log = np.log(timeseries)
    expwighted_avg  = timeseries_log.ewm(halflife = k).mean()
    timeseries_log_ewma_diff = timeseries_log - expwighted_avg 
    timeseries_log_ewma_diff.dropna(inplace=True)
    
    rolling_stat_test(timeseries_log_ewma_diff)
    print("----------")
    adf_test(timeseries_log_ewma_diff)
    print("----------")
    kpss_test(timeseries_log_ewma_diff)
#     return timeseries_log_moving_avg_diff

exponentially_weighted_moving_average(data.Passengers)

----------
Results of Dickey-Fuller Test:
Test Statistic                  -3.601262
p-value                          0.005737
Lags Used                       13.000000
Number of Observations Used    130.000000
Critical Value (1%)             -3.481682
Critical Value (5%)             -2.884042
Critical Value (10%)            -2.578770
dtype: float64
----------
Results of KPSS Test:
Test Statistic            0.310076
p-value                   0.100000
Lags Used                14.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1661: FutureWarning:

The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'

C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning:

p-value is greater than the indicated p-value



### f. Decomposing 
both trend and seasonality are modeled separately

In [40]:
# data['Month'] = pd.to_datetime(data['Month'])
# data= data.set_index('Month')

def decompose_ts(timeseries):
    timeseries_log = np.log(timeseries)
#     decomposition = seasonal_decompose(timeseries,  model='multiplicative')
    decomposition = seasonal_decompose(timeseries_log,  model='multiplicative')

    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    fig = go.Figure(go.Scatter(y = timeseries_log,  mode = 'lines', name="Original"))
    fig.update_layout(title='Original',showlegend=True, template="plotly_white", width=700, height=300)
    fig.show()
    
    fig = go.Figure(go.Scatter(y = trend,  mode = 'lines', name="Trend"))
    fig.update_layout(title='Trend',showlegend=True, template="plotly_white", width=700, height=300)
    fig.show()
    
    fig = go.Figure(go.Scatter(y = seasonal, mode = 'lines', name="Seasonality"))
    fig.update_layout(title='Seasonality',showlegend=True, template="plotly_white", width=700, height=300)
    fig.show()
    
    fig = go.Figure(go.Scatter(y = residual, mode = 'lines', name="Residual"))
    fig.update_layout(title='Residual',showlegend=True, template="plotly_white", width=700, height=300)
    fig.show()
    
    print("----- Stationarity test of Residuals ------")
    adf_test(residual.dropna())
    print("----------")
    kpss_test(residual.dropna())

decompose_ts(data.Passengers)

----- Stationarity test of Residuals ------
Results of Dickey-Fuller Test:
Test Statistic                -6.931283e+00
p-value                        1.082484e-09
Lags Used                      7.000000e+00
Number of Observations Used    1.240000e+02
Critical Value (1%)           -3.484220e+00
Critical Value (5%)           -2.885145e+00
Critical Value (10%)          -2.579359e+00
dtype: float64
----------
Results of KPSS Test:
Test Statistic            0.082892
p-value                   0.100000
Lags Used                13.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1661: FutureWarning:

The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'

C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning:

p-value is greater than the indicated p-value



## 3. Forecasting 

### a. ARIMA
Forecasting for stationary time series is like a linear regression equation. 
The predictors depend on the parameters (p,d,q) of the ARIMA model.

In [ ]:
# Number of AR (Auto-Regressive) terms (p): AR terms are just lags of dependent variable. 
# Number of MA (Moving Average) terms (q): MA terms are lagged forecast errors in prediction equation. i.e difference between the moving average at ith instant and actual value. 
# Number of Differences (d): These are the number of nonseasonal differences.


#### To determine p and q values

#### 1. Autocorrelation Function (ACF)

#### 2. Partial Autocorrelation Function (PACF)

In [43]:
# data.reset_index(level=0, inplace=True)
# data.columns = ['Month', 'Passengers']

def determine_p_q(timeseries, nlags =20):
    
    timeseries_log = np.log(timeseries)
    timeseries_log_diff = timeseries_log - timeseries_log.shift(1)
    lag_acf = acf(timeseries_log_diff.dropna(), nlags=nlags)
    lag_pacf = pacf(timeseries_log_diff.dropna(), nlags=nlags, method='ols')

    fig = go.Figure(go.Scatter(y = lag_acf,  mode = 'lines', name=""))
    fig.update_layout(title='ACF with 95% confidence interval',showlegend=False, width=700, height=400)
    fig.add_shape(type="line",y0=1.96/np.sqrt(len(timeseries_log_diff)),y1=1.96/np.sqrt(len(timeseries_log_diff)),x0=0,x1=20,line=dict(color="LightSeaGreen",width=1.5,dash="dashdot"))
    fig.add_shape(type="line",y0=-1.96/np.sqrt(len(timeseries_log_diff)),y1=-1.96/np.sqrt(len(timeseries_log_diff)),x0=0,x1=20,line=dict(color="LightSeaGreen",width=1.5,dash="dashdot"))
    fig.show()
    
    fig = go.Figure(go.Scatter(y = lag_pacf,  mode = 'lines', name=""))
    fig.update_layout(title='PACF with 95% confidence interval',showlegend=False,  width=700, height=400)
    fig.add_shape(type="line",y0=1.96/np.sqrt(len(timeseries_log_diff)),y1=1.96/np.sqrt(len(timeseries_log_diff)),x0=0,x1=20,line=dict(color="LightSeaGreen",width=1.5,dash="dashdot"))
    fig.add_shape(type="line",y0=-1.96/np.sqrt(len(timeseries_log_diff)),y1=-1.96/np.sqrt(len(timeseries_log_diff)),x0=0,x1=20,line=dict(color="LightSeaGreen",width=1.5,dash="dashdot"))
    fig.show()
    
    return None
    
determine_p_q(data.Passengers)

C:\Users\Asus\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:572: FutureWarning:

fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.



In [44]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [45]:
def arima_ar_model(timeseries):
    timeseries_log =  np.log(timeseries)
    timeseries_log_diff = timeseries_log - timeseries_log.shift(1)
    model = ARIMA(timeseries_log, order=(2, 1, 0))   #(p,d,q)
    model_fit_AR = model.fit(disp=-1) 
    fitted_val = model_fit_AR.fittedvalues
    
    print('RSS: %.4f'% sum((fitted_val-timeseries_log_diff.dropna())**2))
    
    # Taking to original scale
    predictions_ARIMA_diff = pd.Series(model_fit_AR.fittedvalues, copy=True)
    predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
    predictions_ARIMA_log = pd.Series(timeseries_log.iloc[0], index=timeseries_log.index)
    predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
    print("log MAPE: ", mean_absolute_percentage_error(timeseries_log,predictions_ARIMA_log) )
    predictions_ARIMA = np.exp(predictions_ARIMA_log)
    
    print("MAPE: ", mean_absolute_percentage_error(timeseries,predictions_ARIMA) )
    
    fig = go.Figure([go.Scatter(y = timeseries_log_diff, mode = 'lines', name="timeseries_log_diff"),
                    go.Scatter(y = fitted_val, mode = 'lines', name="fittedvalues")])
    fig.update_layout(title='arima_ar_model',showlegend=True, template="plotly_white", width=800, height=300)
    fig.show()
    
    fig = go.Figure([go.Scatter(y = timeseries, mode = 'lines', name="Original"),
                    go.Scatter(y = predictions_ARIMA, mode = 'lines', name="Predicted")])
    fig.update_layout(title='predictions_ARIMA',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
    
    
arima_ar_model(data.Passengers)

RSS: 1.5023
log MAPE:  2.576735166638587
MAPE:  13.120546685504364


In [46]:
def arima_ma_model(timeseries):
    timeseries_log = np.log(timeseries)
    timeseries_log_diff = timeseries_log - timeseries_log.shift(1)
    model = ARIMA(timeseries_log, order=(0, 1, 2))   #(p,d,q)
    model_fit_MA = model.fit(disp=-1) 
    fitted_val = model_fit_MA.fittedvalues
    
    print('RSS: %.4f'% sum((fitted_val-timeseries_log_diff.dropna())**2))
    
    # Taking to original scale
    predictions_ARIMA_diff = pd.Series(model_fit_MA.fittedvalues, copy=True)
    predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
    predictions_ARIMA_log = pd.Series(timeseries_log.iloc[0], index=timeseries_log.index)
    predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
#     print("log MAPE: ", mean_absolute_percentage_error(timeseries_log,predictions_ARIMA_log) )
    predictions_ARIMA = np.exp(predictions_ARIMA_log)
    
    print("MAPE: ", mean_absolute_percentage_error(timeseries,predictions_ARIMA) )
    
    fig = go.Figure([go.Scatter(y = timeseries_log_diff, mode = 'lines', name="timeseries_log_diff"),
                    go.Scatter(y = fitted_val, mode = 'lines', name="fittedvalues")])
    fig.update_layout(title='arima_ma_model',showlegend=True, template="plotly_white", width=800, height=300)
    fig.show()
    
    fig = go.Figure([go.Scatter(y = timeseries, mode = 'lines', name="Original"),
                    go.Scatter(y = predictions_ARIMA, mode = 'lines', name="Predicted")])
    fig.update_layout(title='predictions_ARIMA',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
    
    
arima_ma_model(data.Passengers)

RSS: 1.4721
MAPE:  15.096587176731166


In [47]:
def arima_model(timeseries):
    timeseries_log = np.log(timeseries)
    timeseries_log_diff = timeseries_log - timeseries_log.shift(1)
    
    
    model = ARIMA(timeseries_log, order=(2, 1, 2))   #(p,d,q)
    model_fit = model.fit(disp=-1) 
    fitted_val = model_fit.fittedvalues
    
#     # Model Summary
#     print(model_fit.summary())
    
#     # Residual Error plot
#     residual_err = pd.DataFrame(model_fit.resid)
#     fig = go.Figure(go.Scatter(y = timeseries_log_diff, mode = 'lines', name="residual_err"))
#     fig.update_layout(title='residual_err',showlegend=True, template="plotly_white", width=800, height=400)
#     fig.show()
    
#     residual_err.plot(kind='kde')
#     plt.show()
    
    # fitted value plot
    fig = go.Figure([go.Scatter(y = timeseries_log_diff, mode = 'lines', name="timeseries_log_diff"),
                    go.Scatter(y = fitted_val, mode = 'lines', name="fittedvalues")])
    fig.update_layout(title='arima_model',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
    
    # residual sum of squares (RSS) /sum of squared residuals (SSR)/sum of squared errors of prediction (SSE)
    print('RSS: %.4f'% sum((fitted_val-timeseries_log_diff.dropna())**2))
    
    # Taking to original scale
    predictions_ARIMA_diff = pd.Series(model_fit.fittedvalues, copy=True)
    predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
    predictions_ARIMA_log = pd.Series(timeseries_log.iloc[0], index=timeseries_log.index)
    predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
#     print("log MAPE: ", mean_absolute_percentage_error(timeseries_log,predictions_ARIMA_log) )
    predictions_ARIMA = np.exp(predictions_ARIMA_log)
    
    print("MAPE: ", mean_absolute_percentage_error(timeseries,predictions_ARIMA) )
    # root mean square error
    print('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-timeseries)**2)/len(timeseries)))
    
    
    fig = go.Figure([go.Scatter(y = timeseries, mode = 'lines', name="Original"),
                    go.Scatter(y = predictions_ARIMA, mode = 'lines', name="Predicted")])
    fig.update_layout(title='predictions_ARIMA',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
    
arima_model(data.Passengers)

RSS: 1.0292
MAPE:  25.185315863283193
RMSE: 90.1047


In [48]:
    
#     # Forecast 
#     forecast  = model_fit.forecast(steps=12)[0]
#     forecast = pd.Series(forecast)
#     forecast = forecast.cumsum()
#     forecast = np.exp(forecast)
#     print("Forecast: ",forecast)
    
#     start_index = datetime(1961,1,1)
#     end_index = datetime(1961,12,1)
#     forecast = model_fit.predict(start=start_index, end=end_index)
#     forecast = pd.Series(forecast)
#     forecast = forecast.cumsum()
#     predictions_ARIMA_log = pd.Series(timeseries_log.iloc[-1])
#     predictions_ARIMA_log = predictions_ARIMA_log.add(forecast,fill_value=0)
    
#     forecast = np.exp(predictions_ARIMA_log)
#     print("Forecast: ",forecast)

In [49]:
# Auto Arima 
stepwise_model = auto_arima(data.Passengers, start_p=1, start_q=1,
                           max_p=6, max_q=6, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=1022.896, Time=3.13 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1031.508, Time=0.06 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=1020.393, Time=0.21 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=1021.003, Time=0.18 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=1020.393, Time=0.06 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=1019.239, Time=0.37 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=2.06 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=1020.493, Time=0.33 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=1032.120, Time=0.18 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=1021.120, Time=0.63 sec
 ARIMA(1,1,1)(2,1,0)[12]             : AIC=1021.032, Time=0.55 sec
 ARIMA(0,1,1)(2,1,0)[12]             : AIC=1019.178, Time=0.31 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=1020.425, Time=0.12 sec
 ARIMA(0,1,1)(2,1,1)[12]             : AIC=inf, Time=2.18 sec
 ARIMA(0,1,1)(1,1,1)[12]     

In [53]:
def arima_model_custom(timeseries, p=0,d=1,q=1):
    timeseries_log = np.log(timeseries)
    timeseries_log_diff = timeseries_log - timeseries_log.shift(1)
    
    
    model = ARIMA(timeseries_log, order=(p,d,q))   #(p,d,q)
    model_fit = model.fit(disp=-1) 
    fitted_val = model_fit.fittedvalues
    
#     # Model Summary
#     print(model_fit.summary())
    
#     # Residual Error plot
#     residual_err = pd.DataFrame(model_fit.resid)
#     fig = go.Figure(go.Scatter(y = timeseries_log_diff, mode = 'lines', name="residual_err"))
#     fig.update_layout(title='residual_err',showlegend=True, template="plotly_white", width=800, height=400)
#     fig.show()
    
#     residual_err.plot(kind='kde')
#     plt.show()
    
    # fitted value plot
    fig = go.Figure([go.Scatter(y = timeseries_log_diff, mode = 'lines', name="timeseries_log_diff"),
                    go.Scatter(y = fitted_val, mode = 'lines', name="fittedvalues")])
    fig.update_layout(title='auto_arima_model',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
    
    # residual sum of squares (RSS) /sum of squared residuals (SSR)/sum of squared errors of prediction (SSE)
    print('RSS: %.4f'% sum((fitted_val-timeseries_log_diff.dropna())**2))
    
    # Taking to original scale
    predictions_ARIMA_diff = pd.Series(model_fit.fittedvalues, copy=True)
    predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
    predictions_ARIMA_log = pd.Series(timeseries_log.iloc[0], index=timeseries_log.index)
    predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
#     print("log MAPE: ", mean_absolute_percentage_error(timeseries_log,predictions_ARIMA_log) )
    predictions_ARIMA = np.exp(predictions_ARIMA_log)
    
    print("MAPE: ", mean_absolute_percentage_error(timeseries,predictions_ARIMA) )
    # root mean square error
    print('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-timeseries)**2)/len(timeseries)))
    
    
    fig = go.Figure([go.Scatter(y = timeseries, mode = 'lines', name="Original"),
                    go.Scatter(y = predictions_ARIMA, mode = 'lines', name="Predicted")])
    fig.update_layout(title='predictions_Auto_ARIMA',showlegend=True, template="plotly_white", width=800, height=400)
    fig.show()
    
arima_model_custom(data.Passengers)

RSS: 1.5245
MAPE:  11.423162435165397
RMSE: 50.3861
